In [1]:
# 常用
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import os
# 資料預處理用
import math
import pywt
# 讓每一種標籤的資料都平均一點
import random
# 資料分割用
from sklearn.model_selection import train_test_split
# 機器學習相關
import torch
import torch.utils.data as Data
import torch.nn as nn
import torchvision
import torch.nn.functional as F
# 正規化
from sklearn import preprocessing
# 訓練結果報告分析
from sklearn import metrics
# 儲存報告(使用當前時間作為 pickle 檔名)
import pickle
import time

In [2]:
# 讀取資料的模組
def data_reading(path):
    print('【讀取檔案】')
    data = []
    for file in os.listdir(path):
        data.append(sio.loadmat(path + '/' + file))
        plt.close()
    print('檔案讀取完畢！共讀取了', len(data), '個檔案的資料\n')
    return data
# 讀取受試者資料
def subject_choose(data, subject_ID_list):
    print('【讀取受試者資料】')
    # subject_emg[受試者編號][emg編號][訊號]
    # subject_emg[受試者編號][狀態]
    subject_emg = []
    subject_restimulus = []
    # 選擇受試者
    for subject_ID in subject_ID_list:
        print('受試者', subject_ID, '資料讀取')
        sub_data = list(filter(lambda x: np.squeeze(x['subject']) == subject_ID, data))[0]
        subject_emg.append(sub_data['emg'].T)
        subject_restimulus.append(sub_data['restimulus'])
    print('受試者資料讀取完畢！\n')
    return subject_emg, subject_restimulus

In [3]:
# 資料預處理

def wavelet_with_energy_sum(subject_emg, subject_restimulus, sliding_window_size, sliding_duration, wavelet, mode, maxlevel):
    print('【預處理-小波包能量】')
    x_data = []
    y_data = []
    count = 0
    for emg, restimulus in zip(subject_emg, subject_restimulus):
        count += 1
        print('第' + str(count) + '位受試者資料預處理')
        total_len = len(restimulus)
        # math.ceil -> 無條件進位
        sliding_times = math.ceil((total_len - sliding_window_size) / sliding_duration) + 1

        # 資料分割 + 特徵提取
        window_begin = 0
        for i in range(sliding_times):
        #   特徵提取
            feature_matrix = []
            for e in emg:
                emg_segment = e[window_begin:window_begin+sliding_window_size]
            #   使用多階小波包轉換
            #   小波包基底: db5
            #   層數: 4
                wp = pywt.WaveletPacket(data=emg_segment, wavelet=wavelet, mode=mode, maxlevel=maxlevel)
            #   對第四層每一節點做能量值計算
                wavelet_energy = []
                for j in [node.path for node in wp.get_level(wp.maxlevel, 'natural')]:
                    wavelet_energy.append(np.sum( (np.array(wp[j].data)) ** 2 ))
                feature_matrix.append(wavelet_energy)
            x_data.append(feature_matrix)
        #   標標籤
            restimulus_segment = restimulus[window_begin:window_begin+sliding_window_size]
        #   np.sqeeze()把矩陣內的單維向量的框框消掉
            counts = np.bincount(np.squeeze(restimulus_segment))
            #返回眾數(注意:此方法只有非負數列才可使用)
            label_action_ID = np.argmax(counts)
            y_data.append(label_action_ID)
            window_begin = window_begin + sliding_duration
    print('資料預處理完畢！共', len(x_data), '筆資料\n')
    print('資料標籤數量分布：', np.bincount(np.squeeze(y_data)))
#     讓 label 0 的資料減少一點，使資料分布平均
    x_filter_data = []
    y_filter_data = []
    for i in range(len(x_data)):
        if y_data[i] == 0:
            if random.randint(1, round(np.bincount(np.squeeze(y_data))[0]/np.bincount(np.squeeze(y_data))[1]) ) == 1:
                x_filter_data.append(x_data[i])
                y_filter_data.append(y_data[i])
        else:
            x_filter_data.append(x_data[i])
            y_filter_data.append(y_data[i])
    x_data = x_filter_data
    y_data = y_filter_data
    del x_filter_data
    del y_filter_data
    print('\n資料篩選完成')
    print('資料數量： x -> ', len(x_data), ', y ->', len(y_data))
    print('資料標籤分佈：', np.bincount(np.squeeze(y_data)),'\n')
    # 正規化
    x_data = list(preprocessing.scale(np.array(x_data).reshape(-1)).reshape(-1, 10, 16))
    for i in range(len(x_data)):
        x_data[i] = [x_data[i]]
    return x_data, y_data

def wavelet_parameters_only(subject_emg, subject_restimulus, sliding_window_size, sliding_duration, wavelet, mode,
                            maxlevel):
    print('【預處理-小波包係數】')
    x_data = []
    y_data = []
    count = 0
    # 確保特徵矩陣的 shape 都一樣
    standard_feature_matrix_len1 = 0
    standard_feature_matrix_len2 = 0
    for emg, restimulus in zip(subject_emg, subject_restimulus):
        count += 1
        print('第' + str(count) + '位受試者資料預處理')
        total_len = len(restimulus)
        # math.ceil -> 無條件進位
        sliding_times = math.ceil((total_len - sliding_window_size) / sliding_duration) + 1
        # 資料分割 + 特徵提取
        window_begin = 0
        for i in range(sliding_times):
            #   特徵提取
            feature_matrix = []
            for e in emg:
                emg_segment = e[window_begin:window_begin + sliding_window_size]
                #   使用多階小波包轉換
                #   小波包基底: db5
                #   層數: 4
                wp = pywt.WaveletPacket(data=emg_segment, wavelet=wavelet, mode=mode, maxlevel=maxlevel)
                wavelet_parameter = []
                for j in [node.path for node in wp.get_level(wp.maxlevel, 'natural')]:
                    wavelet_parameter.append(wp[j].data)
                feature_matrix.append(wavelet_parameter)
            #   標標籤
            restimulus_segment = restimulus[window_begin:window_begin + sliding_window_size]
            #   np.sqeeze()把矩陣內的單維向量的框框消掉
            counts = np.bincount(np.squeeze(restimulus_segment))
            # 返回眾數(注意:此方法只有非負數列才可使用)
            label_action_ID = np.argmax(counts)
            # 特徵矩陣滿足標準尺寸（ shape ）才可進資料中
            if i == 0:
                standard_feature_matrix_len1 = len(feature_matrix[0])
                standard_feature_matrix_len2 = len(feature_matrix[0][0])
            if len(feature_matrix[0]) == standard_feature_matrix_len1 and len(feature_matrix[0][0]) == standard_feature_matrix_len2:
                x_data.append(feature_matrix)
                y_data.append(label_action_ID)
            window_begin = window_begin + sliding_duration
    print('資料預處理完畢！共', len(x_data), '筆資料\n')
    print('資料標籤數量分布：', np.bincount(np.squeeze(y_data)))
    #     讓 label 0 的資料減少一點，使資料分布平均
    x_filter_data = []
    y_filter_data = []
    for i in range(len(x_data)):
        if y_data[i] == 0:
            if random.randint(1, round(np.bincount(np.squeeze(y_data))[0] / np.bincount(np.squeeze(y_data))[1])) == 1:
                x_filter_data.append(x_data[i])
                y_filter_data.append(y_data[i])
        else:
            x_filter_data.append(x_data[i])
            y_filter_data.append(y_data[i])
    x_data = x_filter_data
    y_data = y_filter_data
    del x_filter_data
    del y_filter_data
    print('\n資料篩選完成')
    print('資料數量： x -> ', len(x_data), ', y ->', len(y_data))
    print('資料標籤分佈：', np.bincount(np.squeeze(y_data)), '\n')
    # 正規化
    print('正規化')
    print(np.array(x_data).shape)
    x_data = list(preprocessing.scale(np.array(x_data).reshape(-1)).reshape(-1, 10, standard_feature_matrix_len1, standard_feature_matrix_len2))
    return x_data, y_data, standard_feature_matrix_len1, standard_feature_matrix_len2

In [4]:
# 機器學習模組

# 資料集設定
def data_setting(x_data, y_data, BATCH_SIZE):
    print('\n【資料集設定】\n')
    # 先转换成 torch 能识别的 Dataset
    torch_dataset = Data.TensorDataset(torch.FloatTensor(x_data), torch.LongTensor(y_data))

    # 把 dataset 放入 DataLoader
    loader = Data.DataLoader(
        dataset=torch_dataset,      # torch TensorDataset format
        batch_size=BATCH_SIZE,      # mini batch size
        shuffle=True,               # 要不要打乱数据 (打乱比较好)
    )
    return loader
# CNN_energy
class CNN_energy(nn.Module):
    def __init__(self):
        super(CNN_energy, self).__init__()
        # nn.Sequential()可以快速搭建神經網路
        # 卷積運算所使用的mask一般稱為kernel map，這邊為5x5的map
        # stride決定kernel map一次要走幾格
        # 上面用5x5的kernel map去跑28x28的圖片，卷積完會只剩下26x26，故加兩層
        # zero-padding 讓圖片大小相同
        self.conv1 = nn.Sequential( # input shape(channel=1, height=28, weight=28)
            nn.Conv2d(
                in_channels = 1, #  輸入信號的通道
                out_channels = 4, # 卷積產生的通道
                kernel_size = (3, 5), # 卷積核的尺寸
                stride = 1, # 卷積步長
                padding = (1,2) # 輸入的每一條邊補充0的層數
            ),  # output shape(4, 10, 16)
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            # input shape(4, 10, 16)
            nn.Conv2d(4, 8, (3,5), 1, (1,2)), # output shape(8, 10, 16)
            nn.ReLU()
        )
        self.hidden = nn.Linear(8*10*16, 800)
        self.hidden2 = nn.Linear(800, 800)
        self.hidden3 = nn.Linear(800, 800)
        self.out = nn.Linear(800, 13) # fully connected layer
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # 展平多維卷積圖層 (batch_size, 32*10*16)
        x = self.hidden(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = F.softmax(self.out(x))
        return output
# CNN_parameter
class CNN_parameter(nn.Module):
    def __init__(self, CNN_neurons_num, standard_feature_matrix_len1, standard_feature_matrix_len2):
        super(CNN_parameter,self).__init__()
        # nn.Sequential()可以快速搭建神經網路
        # 卷積運算所使用的mask一般稱為kernel map，這邊為5x5的map
        # stride決定kernel map一次要走幾格
        # 上面用5x5的kernel map去跑28x28的圖片，卷積完會只剩下26x26，故加兩層
        # zero-padding 讓圖片大小相同
        self.conv1 = nn.Sequential(  # input shape(channel=1, height=28, weight=28)
            nn.Conv2d(
                in_channels=10,  # 輸入信號的通道
                out_channels=20,  # 卷積產生的通道
                kernel_size=(3, 5),  # 卷積核的尺寸
                stride=1,  # 卷積步長
                padding=(1, 2)  # 輸入的每一條邊補充0的層數
            ),  # output shape(20, 16, 20)
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            # input shape(4, 10, 16)
            nn.Conv2d(20, 40, (3, 5), 1, (1, 2)),  # output shape(40, 16, 20)
            nn.ReLU()
        )
        self.hidden = nn.Linear(40 * standard_feature_matrix_len1 * standard_feature_matrix_len2, CNN_neurons_num)
        self.hidden2 = nn.Linear(CNN_neurons_num, CNN_neurons_num)
        self.hidden3 = nn.Linear(CNN_neurons_num, CNN_neurons_num)
        self.out = nn.Linear(CNN_neurons_num, 13)  # fully connected layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # 展平多維卷積圖層 (batch_size, 32*10*16)
        x = self.hidden(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        output = F.softmax(self.out(x))
        return output
# 訓練模型
def module_training(loader, EPOCH, LR, module_class, standard_feature_matrix_len1,standard_feature_matrix_len2):
    print('【訓練模型】')
    module = module_class(800, standard_feature_matrix_len1, standard_feature_matrix_len2)
    print(module)
    # optimize all cnn parameters
    optimizer = torch.optim.Adam(module.parameters(), lr=LR)
    # the target label is not one-hotted
    # pytorch 的 CrossEntropyLoss 會自動把張量轉為 one hot形式
    loss_func = nn.CrossEntropyLoss()

    # training and testing
    for epoch in range(EPOCH):
        print('第', epoch, '次訓練')
        # enumerate : 枚舉可列舉對象．ex.
        # A = [a, b, c]
        # list(enumerate(A)) = [(0,a), (1,b), (2,c)]
        for step, (b_x, b_y) in enumerate(loader):
            output = module(b_x)
            loss = loss_func(output, b_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (step % 50 == 0):
                print('step:' + str(step))
                print('loss:' + str(loss))
    print('訓練完成！！\n')
    return module

In [5]:
# 訓練結果分析
def testing_report(module, testing_x_data, testing_y_data):
    print('【訓練結果報告】')
    test_output = module(torch.FloatTensor(testing_x_data))
    pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
    # 混淆矩陣
    confusion_matrix = metrics.confusion_matrix(testing_y_data,pred_y)
    print('混淆矩陣')
    print('true/predict')
    print(confusion_matrix)
    classification_report = metrics.classification_report(testing_y_data,pred_y)
    print('\n<============================>\n\n分類報告')
    print(classification_report)

    print('<============================>\n\n分類準確率')
    correct = 0
    for i in range(len(testing_y_data)):
        if testing_y_data[i] == pred_y[i]:
            correct += 1
    print('測試資料數：', len(testing_y_data), ', 預測正確數：', correct, '準確率：', (correct/len(testing_y_data)))
    return confusion_matrix, classification_report, (correct/len(testing_y_data))

In [6]:
feature_extract_method = {'wavelet_with_energy_sum':wavelet_with_energy_sum, 'wavelet_parameters_only':wavelet_parameters_only}
CNN_module = {'wavelet_with_energy_sum':CNN_energy, 'wavelet_parameters_only':CNN_parameter}

def do_one_training(epoch, batch_size, CNN_neuron_num, sliding_window_size, sliding_window_movement, feature_extract, learning_rate, subject):
    data = data_reading('S_All_A1_E1')
    subject_emg, subject_restimulus = subject_choose(data, subject)
    x_data, y_data = feature_extract_method[feature_extract](subject_emg, subject_restimulus, sliding_window_size, sliding_window_movement, 'db5', 'symmetric', 4)
    # 將資料拆分成訓練與測試集
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, random_state = 0)
    print('\n資料拆分完成')
    print('訓練資料: x -> ', len(x_train), ', y -> ', len(y_train))
    print('測試資料: x -> ', len(x_test), ', y -> ', len(y_test))
    loader = data_setting(x_train, y_train, batch_size)
    module = module_training(loader, epoch, learning_rate, CNN_module[feature_extract])
    print('\n訓練資料結果分析\n')
    confusion_matrix_train, classification_report_train, accuracy_train = testing_report(module, x_train, y_train)
    print('\n測試資料結果分析\n')
    confusion_matrix_test, classification_report_test, accuracy_test = testing_report(module, x_test, y_test)
    test_report_dict = {'epoch':epoch,
                        'batch_size':batch_size, 
                        'module':module, 'sliding_window_size':sliding_window_size, 
                        'sliding_window_movement':sliding_window_movement, 
                        'feature_extract':feature_extract, 
                        'learning_rate':learning_rate, 
                        'subject':subject, 
                        'confusion_matrix_train':confusion_matrix_train, 
                        'classification_report_train':classification_report_train,
                        'accuracy_train': accuracy_train, 
                        'confusion_matrix_test':confusion_matrix_test, 
                        'classification_report_test':classification_report_test,
                        'accuracy_test': accuracy_test}
#     path = 'training_report/DB1_training_report_' + time.strftime("%Y%m%d_%H%M%S", time.localtime())
    path = 'training_report/DB1_training_report_' + time.strftime("%Y%m%d_%H%M%S", time.localtime()) + '.pickle'
    file = open(path, 'wb')
    pickle.dump(test_report_dict, file)
    file.close()
    print('\n訓練完畢! 結果報告已匯入至 ' + path)

In [9]:
data = data_reading('S_All_A1_E1')
subject = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
# subject = [1]
feature_extract = 'wavelet_parameters_only'
#sliding_window_size_list = [80, 100, 200, 300, 400, 500]
max_level_list = [2,3,4,5,6]
sliding_window_size = 80
sliding_window_movement = 40
batch_size = 2048
learning_rate = 0.0005
# epoch_list = [10, 20 ,30, 40, 50, 60, 70 ,80, 90, 100, 110, 120, 130, 140, 150]
epoch = 100
subject_emg, subject_restimulus = subject_choose(data, subject)
for max_level in max_level_list:
    sliding_window_movement = int(sliding_window_size / 2)
    x_data, y_data,standard_feature_matrix_len1,standard_feature_matrix_len2 = feature_extract_method[feature_extract](subject_emg, subject_restimulus, sliding_window_size,
                                                             sliding_window_movement, 'db5', 'symmetric', max_level)
    # 將資料拆分成訓練與測試集
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=0)
    print('\n資料拆分完成')
    print('訓練資料: x -> ', len(x_train), ', y -> ', len(y_train))
    print('測試資料: x -> ', len(x_test), ', y -> ', len(y_test))
    loader = data_setting(x_train, y_train, batch_size)
    module = module_training(loader, epoch, learning_rate, CNN_module['wavelet_parameters_only'], standard_feature_matrix_len1,standard_feature_matrix_len2)
    print('\n訓練資料結果分析\n')
    confusion_matrix_train, classification_report_train, accuracy_train = testing_report(module, x_train, y_train)
    print('\n測試資料結果分析\n')
    confusion_matrix_test, classification_report_test, accuracy_test = testing_report(module, x_test, y_test)
    test_report_dict = {'epoch': epoch,
                        'batch_size': batch_size,
                        'module': module, 'sliding_window_size': sliding_window_size,
                        'sliding_window_movement': sliding_window_movement,
                        'feature_extract': feature_extract,
                        'learning_rate': learning_rate,
                        'subject': subject,
                        'confusion_matrix_train': confusion_matrix_train,
                        'classification_report_train': classification_report_train,
                        'accuracy_train': accuracy_train,
                        'confusion_matrix_test': confusion_matrix_test,
                        'classification_report_test': classification_report_test,
                        'accuracy_test': accuracy_test}
    #     path = 'training_report/DB1_training_report_' + time.strftime("%Y%m%d_%H%M%S", time.localtime())
    path = 'training_reprot_wavelet_level/DB1_training_report_' + time.strftime("%Y%m%d_%H%M%S", time.localtime()) + '.pickle'
    file = open(path, 'wb')
    pickle.dump(test_report_dict, file)
    file.close()
    print('\n訓練完畢! 結果報告已匯入至 ' + path)

【讀取檔案】
檔案讀取完畢！共讀取了 27 個檔案的資料

【讀取受試者資料】
受試者 1 資料讀取
受試者 2 資料讀取
受試者 3 資料讀取
受試者 4 資料讀取
受試者 5 資料讀取
受試者 6 資料讀取
受試者 7 資料讀取
受試者 8 資料讀取
受試者 9 資料讀取
受試者 10 資料讀取
受試者 11 資料讀取
受試者 12 資料讀取
受試者 13 資料讀取
受試者 14 資料讀取
受試者 15 資料讀取
受試者 16 資料讀取
受試者 17 資料讀取
受試者 18 資料讀取
受試者 19 資料讀取
受試者 20 資料讀取
受試者 21 資料讀取
受試者 22 資料讀取
受試者 23 資料讀取
受試者 24 資料讀取
受試者 25 資料讀取
受試者 26 資料讀取
受試者 27 資料讀取
受試者資料讀取完畢！

【預處理-小波包係數】
第1位受試者資料預處理
第2位受試者資料預處理
第3位受試者資料預處理
第4位受試者資料預處理
第5位受試者資料預處理
第6位受試者資料預處理
第7位受試者資料預處理
第8位受試者資料預處理
第9位受試者資料預處理
第10位受試者資料預處理
第11位受試者資料預處理
第12位受試者資料預處理
第13位受試者資料預處理
第14位受試者資料預處理
第15位受試者資料預處理
第16位受試者資料預處理
第17位受試者資料預處理
第18位受試者資料預處理
第19位受試者資料預處理
第20位受試者資料預處理
第21位受試者資料預處理
第22位受試者資料預處理
第23位受試者資料預處理
第24位受試者資料預處理
第25位受試者資料預處理
第26位受試者資料預處理
第27位受試者資料預處理
資料預處理完畢！共 68247 筆資料

資料標籤數量分布： [38611  2569  2517  2830  2373  2374  2477  2494  2615  2386  2301  2213
  2487]

資料篩選完成
資料數量： x ->  32241 , y -> 32241
資料標籤分佈： [2605 2569 2517 2830 2373 2374 2477 2494 2615 2386 2301 2213 2487] 

正規化
(32241, 10, 4, 26)

資料拆分完成
訓練資料: x ->  22568 , 

C:\Users\ericFang\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step:0
loss:tensor(2.5649, grad_fn=<NllLossBackward>)
第 1 次訓練
step:0
loss:tensor(2.4780, grad_fn=<NllLossBackward>)
第 2 次訓練
step:0
loss:tensor(2.4369, grad_fn=<NllLossBackward>)
第 3 次訓練
step:0
loss:tensor(2.4197, grad_fn=<NllLossBackward>)
第 4 次訓練
step:0
loss:tensor(2.4100, grad_fn=<NllLossBackward>)
第 5 次訓練
step:0
loss:tensor(2.3893, grad_fn=<NllLossBackward>)
第 6 次訓練
step:0
loss:tensor(2.4177, grad_fn=<NllLossBackward>)
第 7 次訓練
step:0
loss:tensor(2.4103, grad_fn=<NllLossBackward>)
第 8 次訓練
step:0
loss:tensor(2.3907, grad_fn=<NllLossBackward>)
第 9 次訓練
step:0
loss:tensor(2.3649, grad_fn=<NllLossBackward>)
第 10 次訓練
step:0
loss:tensor(2.3832, grad_fn=<NllLossBackward>)
第 11 次訓練
step:0
loss:tensor(2.4113, grad_fn=<NllLossBackward>)
第 12 次訓練
step:0
loss:tensor(2.3737, grad_fn=<NllLossBackward>)
第 13 次訓練
step:0
loss:tensor(2.3857, grad_fn=<NllLossBackward>)
第 14 次訓練
step:0
loss:tensor(2.3712, grad_fn=<NllLossBackward>)
第 15 次訓練
step:0
loss:tensor(2.3433, grad_fn=<NllLossBackward>)
第 16 次訓練
s

C:\Users\ericFang\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



測試資料結果分析

【訓練結果報告】
混淆矩陣
true/predict
[[526   0  66   0   0   9  17  56   0   0  71  25  16]
 [ 73   0 187   0   0 115 142  59   0   0  41 143   2]
 [ 96   0 426   0   0  43  72   6   0   0  35  88   3]
 [142   0 140   0   0 154 155  66   0   0  80 102  12]
 [ 58   0  89   0   0  58 242  71   0   0  19 140   7]
 [ 55   0  49   0   0 401  50  70   0   0  22  82   3]
 [ 54   0  93   0   0  24 443  30   0   0  18  64   1]
 [ 66   0  97   0   0 109 264 115   0   0   7 131   2]
 [197   0  85   0   0  56 149  50   0   0  43 198  11]
 [102   0 171   0   0  69  61  14   0   0  45 236  17]
 [199   0  80   0   0   9  41  11   0   0 199 129  20]
 [ 74   0 185   0   0  23  11   2   0   0  35 298   7]
 [194   0 116   0   0 117  61   4   0   0  69 152  31]]

<============================>

分類報告
              precision    recall  f1-score   support

           0       0.29      0.67      0.40       786
           1       0.00      0.00      0.00       762
           2       0.24      0.55      0.33  

...

C:\Users\ericFang\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



<============================>

分類報告
              precision    recall  f1-score   support

           0       0.32      0.67      0.43      1806
           1       0.26      0.41      0.32      1801
           2       0.31      0.72      0.43      1761
           3       0.18      0.40      0.25      1991
           4       0.45      0.44      0.44      1627
           5       0.00      0.00      0.00      1687
           6       0.39      0.45      0.42      1748
           7       0.00      0.00      0.00      1736
           8       0.00      0.00      0.00      1795
           9       0.00      0.00      0.00      1650
          10       0.30      0.36      0.33      1628
          11       0.39      0.49      0.43      1559
          12       0.00      0.00      0.00      1763

    accuracy                           0.30     22552
   macro avg       0.20      0.30      0.24     22552
weighted avg       0.20      0.30      0.23     22552

<============================>

分類準確率
測試資

C:\Users\ericFang\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step:0
loss:tensor(2.5650, grad_fn=<NllLossBackward>)
第 1 次訓練
step:0
loss:tensor(2.4676, grad_fn=<NllLossBackward>)
第 2 次訓練
step:0
loss:tensor(2.4212, grad_fn=<NllLossBackward>)
第 3 次訓練
step:0
loss:tensor(2.3944, grad_fn=<NllLossBackward>)
第 4 次訓練
step:0
loss:tensor(2.3839, grad_fn=<NllLossBackward>)
第 5 次訓練
step:0
loss:tensor(2.3468, grad_fn=<NllLossBackward>)
第 6 次訓練
step:0
loss:tensor(2.3334, grad_fn=<NllLossBackward>)
第 7 次訓練
step:0
loss:tensor(2.3487, grad_fn=<NllLossBackward>)
第 8 次訓練
step:0
loss:tensor(2.3321, grad_fn=<NllLossBackward>)
第 9 次訓練
step:0
loss:tensor(2.3061, grad_fn=<NllLossBackward>)
第 10 次訓練
step:0
loss:tensor(2.2926, grad_fn=<NllLossBackward>)
第 11 次訓練
step:0
loss:tensor(2.3056, grad_fn=<NllLossBackward>)
第 12 次訓練
step:0
loss:tensor(2.3045, grad_fn=<NllLossBackward>)
第 13 次訓練
step:0
loss:tensor(2.2783, grad_fn=<NllLossBackward>)
第 14 次訓練
step:0
loss:tensor(2.2797, grad_fn=<NllLossBackward>)
第 15 次訓練
step:0
loss:tensor(2.2609, grad_fn=<NllLossBackward>)
第 16 次訓練
s

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 17289984000 bytes. Buy new RAM!
